In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

Generate the Dataset

In [2]:
def generate_time_series(sample_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, sample_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))  #   wave 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + wave 2
    series += 0.1 * (np.random.rand(sample_size, n_steps) - 0.5)   # + noise
    return series[..., np.newaxis].astype(np.float32)

In [3]:
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

In [19]:
X_train.shape, y_train.shape

((7000, 50, 1), (7000, 1))

Fully Connected Neural Network Model

In [17]:
fcnn_model = keras.models.Sequential([
    keras.layers.Input(shape=[50, 1]),
    keras.layers.Flatten(),
    keras.layers.Dense(1)
])

fcnn_model.compile(loss="mse", optimizer="adam")

In [9]:
history = fcnn_model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1320 - val_loss: 0.0238
Epoch 2/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0210 - val_loss: 0.0125
Epoch 3/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0130 - val_loss: 0.0101
Epoch 4/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0101 - val_loss: 0.0088
Epoch 5/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0093 - val_loss: 0.0078
Epoch 6/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0078 - val_loss: 0.0070
Epoch 7/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0074 - val_loss: 0.0064
Epoch 8/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0065 - val_loss: 0.0060
Epoch 9/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0062 - val_loss: 0.0056
Epoch 10/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0057 - val_loss: 0.0052
Epoch 11/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0054 - val_loss: 0.0050
Epoch 12/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

Simple RNN Model

In [18]:
rnn_model = keras.models.Sequential([
    keras.layers.Input(shape=[50, 1]),
    keras.layers.SimpleRNN(10, return_sequences=True),
    keras.layers.SimpleRNN(20)
])
rnn_model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn_11 (SimpleRNN)       │ (None, 50, 10)         │           120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_12 (SimpleRNN)       │ (None, 20)             │           620 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 740 (2.89 KB)

 Trainable params: 740 (2.89 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
optimizer = keras.optimizers.Adam(learning_rate=0.005)
rnn_model.compile(loss="mse", optimizer=optimizer)

In [16]:
history = rnn_model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/5
219/219 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.0036 - val_loss: 0.0035
Epoch 2/5
219/219 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - loss: 0.0037 - val_loss: 0.0040
Epoch 3/5
219/219 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - loss: 0.0037 - val_loss: 0.0044
Epoch 4/5
219/219 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 5/5
219/219 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 0.0036 - val_loss: 0.0036


Deep RNN

In [24]:
drnn_model = keras.models.Sequential([
    keras.layers.Input(shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(1)
])
drnn_model.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn_19 (SimpleRNN)       │ (None, 50, 20)         │           440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_20 (SimpleRNN)       │ (None, 50, 20)         │           820 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_21 (SimpleRNN)       │ (None, 1)              │            22 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,282 (5.01 KB)

 Trainable params: 1,282 (5.01 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
drnn_model.compile(loss="mse", optimizer="adam")
history = drnn_model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/5
219/219 ━━━━━━━━━━━━━━━━━━━━ 10s 30ms/step - loss: 0.0048 - val_loss: 0.0034
Epoch 2/5
219/219 ━━━━━━━━━━━━━━━━━━━━ 7s 31ms/step - loss: 0.0032 - val_loss: 0.0033
Epoch 3/5
219/219 ━━━━━━━━━━━━━━━━━━━━ 10s 30ms/step - loss: 0.0032 - val_loss: 0.0030
Epoch 4/5
219/219 ━━━━━━━━━━━━━━━━━━━━ 9s 26ms/step - loss: 0.0031 - val_loss: 0.0033
Epoch 5/5
219/219 ━━━━━━━━━━━━━━━━━━━━ 10s 25ms/step - loss: 0.0030 - val_loss: 0.0028
